# Kaggle March Madness Competition

In [1]:
from pathlib import Path
import os
import pandas as pd

In [2]:
# set path to data directories
raw_data = Path('../march-machine-learning-mania-2023/')
data_path = Path('../data/')

## Load Relevant Raw Data

In [3]:
# teams
m_teams = pd.read_csv(raw_data/'MTeams.csv')
w_teams = pd.read_csv(raw_data/'WTeams.csv')

# seasons
m_seasons = pd.read_csv(raw_data/'MSeasons.csv')
w_seasons = pd.read_csv(raw_data/'WSeasons.csv')

# NCAA tournament seeds
m_seeds = pd.read_csv(raw_data/'MNCAATourneySeeds.csv')
w_seeds = pd.read_csv(raw_data/'WNCAATourneySeeds.csv')

# compact regular season results
m_season_compact = pd.read_csv(raw_data/'MRegularSeasonCompactResults.csv')
w_season_compact = pd.read_csv(raw_data/'WRegularSeasonCompactResults.csv')

# compact tournament results
m_ncaa_compact = pd.read_csv(raw_data/'MNCAATourneyCompactResults.csv')
w_ncaa_compact = pd.read_csv(raw_data/'WNCAATourneyCompactResults.csv')

# sample submission file
samp_submission = pd.read_csv(raw_data/'SampleSubmission2023.csv')

## Preprocess Data

In [4]:
# concat mens and womens datasets
teams = pd.concat([m_teams, w_teams], axis=0)
seasons = pd.concat([m_seasons, w_seasons], axis=0)
seeds = pd.concat([m_seeds, w_seeds], axis=0)
season_results = pd.concat([m_season_compact, w_season_compact], axis=0)
ncaa_results = pd.concat([m_ncaa_compact, w_ncaa_compact])

In [5]:
# add winning team seeds to ncaa_results
ncaa_results = ncaa_results.merge(seeds, how='left', left_on=['WTeamID', 'Season'], right_on=['TeamID', 'Season'])
ncaa_results.rename(columns={'Seed': 'WTeamSeed'}, inplace=True)
ncaa_results.drop('TeamID', axis=1, inplace=True)

# add losing team seeds
ncaa_results = ncaa_results.merge(seeds, how='left', left_on=['LTeamID', 'Season'], right_on=['TeamID', 'Season'])
ncaa_results.rename(columns={'Seed':'LTeamSeed'}, inplace=True)
ncaa_results.drop('TeamID', axis=1, inplace=True)

In [6]:
# rename season columns
seasons.columns = ['Season', 'SeasonStart', 'RegionW', 'RegionX', 'RegionY', 'RegionZ']

# add to ncaa_results
ncaa_results = ncaa_results.merge(seasons, how='left', on='Season')

In [7]:
# identify team regions based on seed
ncaa_results['WTeamRegion'] = ncaa_results['WTeamSeed'].apply(lambda x: x[0])
ncaa_results['LTeamRegion'] = ncaa_results['LTeamSeed'].apply(lambda x: x[0])

In [8]:
# create mapping dict for regions
ncaa_results['mapping_dict'] = ncaa_results.apply(lambda x: {'W': x['RegionW'], 
                                                             'X': x['RegionX'],
                                                             'Y': x['RegionY'],
                                                             'Z': x['RegionZ']}, axis=1)

In [9]:
# replace region codes with the proper region name
ncaa_results['WTeamRegion'] = ncaa_results.apply(lambda x: x['mapping_dict'][x['WTeamRegion']], axis=1)
ncaa_results['LTeamRegion'] = ncaa_results.apply(lambda x: x['mapping_dict'][x['LTeamRegion']], axis=1)

In [10]:
# drop unnecessary columns
ncaa_results.drop(['RegionW', 'RegionX', 'RegionY', 'RegionZ', 'mapping_dict'], axis=1, inplace=True)

In [11]:
# winning team details 
ncaa_results = ncaa_results.merge(teams, how='left', left_on='WTeamID', right_on='TeamID')
ncaa_results.rename(columns={'TeamName': 'WTeamName',
                             'FirstD1Season': 'WTeamFirstD1Season',
                             'LastD1Season': 'WTeamLastD1Season'}, inplace=True)
ncaa_results.drop('TeamID', axis=1, inplace=True)

# losing team details 
ncaa_results = ncaa_results.merge(teams, how='left', left_on='LTeamID', right_on='TeamID')
ncaa_results.rename(columns={'TeamName':'LTeamName',
                             'FirstD1Season': 'LTeamFirstD1Season',
                             'LastD1Season': 'LTeamLastD1Season'}, inplace=True)
ncaa_results.drop('TeamID', axis=1, inplace=True)

In [12]:
# add columns 
season_results['WTeamScoreDiff'] = season_results['WScore'] - season_results['LScore']
season_results['LTeamScoreDiff'] = season_results['LScore'] - season_results['WScore']

In [13]:
# regular season games won
grouped = season_results.groupby(['Season', 'WTeamID']).count().reset_index()
grouped = grouped[['Season', 'WTeamID', 'WScore']]
grouped.columns = ['Season', 'TeamID', 'GamesWon']

# concat into ncaa_results on WTeam
ncaa_results = ncaa_results.merge(grouped, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])
ncaa_results.drop('TeamID', axis=1, inplace=True)
ncaa_results.rename(columns={'GamesWon':'WTeamGamesWon'}, inplace=True)

# concat into ncaa_results on LTeam
ncaa_results = ncaa_results.merge(grouped, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'])
ncaa_results.drop('TeamID', axis=1, inplace=True)
ncaa_results.rename(columns={'GamesWon':'LTeamGamesWon'}, inplace=True)

In [14]:
# regular season games lost
grouped = season_results.groupby(['Season', 'LTeamID']).count().reset_index()
grouped = grouped[['Season', 'LTeamID', 'LScore']]
grouped.columns = ['Season', 'TeamID', 'GamesLost']

# concat into ncaa_results on WTeam
ncaa_results = ncaa_results.merge(grouped, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])
ncaa_results.drop('TeamID', axis=1, inplace=True)
ncaa_results.rename(columns={'GamesLost':'WTeamGamesLost'}, inplace=True)

# concat into ncaa_results on LTeam
ncaa_results = ncaa_results.merge(grouped, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'])
ncaa_results.drop('TeamID', axis=1, inplace=True)
ncaa_results.rename(columns={'GamesLost':'LTeamGamesLost'}, inplace=True)

In [15]:
# average regular season point differential
grouped_win = season_results.groupby(['Season', 'WTeamID']).mean(numeric_only=True).reset_index()
grouped_lose = season_results.groupby(['Season', 'LTeamID']).mean(numeric_only=True).reset_index()

# Win score diff
grouped_win = grouped_win[['Season', 'WTeamID', 'WTeamScoreDiff']]
grouped_win.columns = ['Season', 'TeamID', 'MeanWinScoreDiff']

# Lose score diff
grouped_lose = grouped_lose[['Season', 'LTeamID', 'LTeamScoreDiff']]
grouped_lose.columns = ['Season', 'TeamID', 'MeanLoseScoreDiff']

In [16]:
# concat winning team meanwinscorediff
ncaa_results = ncaa_results.merge(grouped_win, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])
ncaa_results.drop('TeamID', axis=1, inplace=True)
ncaa_results.rename(columns={'MeanWinScoreDiff':'WTeamMeanWinDiff'}, inplace=True)

# concat losing team meanwinscorediff
ncaa_results = ncaa_results.merge(grouped_win, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'])
ncaa_results.drop('TeamID', axis=1, inplace=True)
ncaa_results.rename(columns={'MeanWinScoreDiff':'LTeamMeanWinDiff'}, inplace=True)

In [17]:
# concat winning team mean lose score diff
ncaa_results = ncaa_results.merge(grouped_lose, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])
ncaa_results.drop('TeamID', axis=1, inplace=True)
ncaa_results.rename(columns={'MeanLoseScoreDiff':'WTeamMeanLoseDiff'}, inplace=True)

# concat losing team mean lose score diff
ncaa_results = ncaa_results.merge(grouped_lose, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'])
ncaa_results.drop('TeamID', axis=1, inplace=True)
ncaa_results.rename(columns={'MeanLoseScoreDiff':'LTeamMeanLoseDiff'}, inplace=True)

In [18]:
ncaa_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WTeamSeed,LTeamSeed,...,LTeamFirstD1Season,LTeamLastD1Season,WTeamGamesWon,LTeamGamesWon,WTeamGamesLost,LTeamGamesLost,WTeamMeanWinDiff,LTeamMeanWinDiff,WTeamMeanLoseDiff,LTeamMeanLoseDiff
0,1985,136,1116,63,1234,54,N,0,X09,X08,...,1985.0,2023.0,21,20,12.0,10.0,10.333333,18.450000,-8.083333,-5.500000
1,1985,136,1120,59,1345,58,N,0,Z11,Z06,...,1985.0,2023.0,18,17,11.0,8.0,11.833333,12.529412,-9.636364,-14.750000
2,1985,136,1207,68,1250,43,N,0,W01,W16,...,1985.0,2023.0,25,11,2.0,18.0,17.040000,6.000000,-1.500000,-10.833333
3,1985,136,1229,58,1425,55,N,0,Y09,Y08,...,1985.0,2023.0,20,19,7.0,9.0,11.350000,9.421053,-9.428571,-8.111111
4,1985,136,1242,49,1325,38,N,0,Z03,Z14,...,1985.0,2023.0,23,20,7.0,7.0,10.043478,8.800000,-8.857143,-7.571429


In [19]:
ncaa_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WTeamSeed,LTeamSeed,...,LTeamFirstD1Season,LTeamLastD1Season,WTeamGamesWon,LTeamGamesWon,WTeamGamesLost,LTeamGamesLost,WTeamMeanWinDiff,LTeamMeanWinDiff,WTeamMeanLoseDiff,LTeamMeanLoseDiff
0,1985,136,1116,63,1234,54,N,0,X09,X08,...,1985.0,2023.0,21,20,12.0,10.0,10.333333,18.450000,-8.083333,-5.500000
1,1985,136,1120,59,1345,58,N,0,Z11,Z06,...,1985.0,2023.0,18,17,11.0,8.0,11.833333,12.529412,-9.636364,-14.750000
2,1985,136,1207,68,1250,43,N,0,W01,W16,...,1985.0,2023.0,25,11,2.0,18.0,17.040000,6.000000,-1.500000,-10.833333
3,1985,136,1229,58,1425,55,N,0,Y09,Y08,...,1985.0,2023.0,20,19,7.0,9.0,11.350000,9.421053,-9.428571,-8.111111
4,1985,136,1242,49,1325,38,N,0,Z03,Z14,...,1985.0,2023.0,23,20,7.0,7.0,10.043478,8.800000,-8.857143,-7.571429


In [20]:
list(ncaa_results.columns)

['Season',
 'DayNum',
 'WTeamID',
 'WScore',
 'LTeamID',
 'LScore',
 'WLoc',
 'NumOT',
 'WTeamSeed',
 'LTeamSeed',
 'SeasonStart',
 'WTeamRegion',
 'LTeamRegion',
 'WTeamName',
 'WTeamFirstD1Season',
 'WTeamLastD1Season',
 'LTeamName',
 'LTeamFirstD1Season',
 'LTeamLastD1Season',
 'WTeamGamesWon',
 'LTeamGamesWon',
 'WTeamGamesLost',
 'LTeamGamesLost',
 'WTeamMeanWinDiff',
 'LTeamMeanWinDiff',
 'WTeamMeanLoseDiff',
 'LTeamMeanLoseDiff']

In [21]:
# drop unnecessary columns
ncaa_results = ncaa_results.drop(['WLoc', 'NumOT', 'SeasonStart', 'WTeamRegion', 'LTeamRegion',
                                  'WTeamFirstD1Season', 'WTeamLastD1Season',
                                  'LTeamFirstD1Season', 'LTeamLastD1Season'], axis=1)

In [22]:
# create copy and drop additional columns
preprocessed = ncaa_results.copy() 
preprocessed = preprocessed.drop(['WScore', 'LScore', 'WTeamName', 'LTeamName', 'DayNum'], axis=1)

In [23]:
# pull out seed numbers
import re
preprocessed['WTeamSeed'] = preprocessed['WTeamSeed'].apply(lambda x: re.sub('[^0-9]', '', x))
preprocessed['LTeamSeed'] = preprocessed['LTeamSeed'].apply(lambda x: re.sub('[^0-9]', '', x))

In [24]:
# preview sample submission
samp_submission.head()

,ID,Pred
0,2023_1101_1102,0.5
1,2023_1101_1103,0.5
2,2023_1101_1104,0.5
3,2023_1101_1105,0.5
4,2023_1101_1106,0.5


In [25]:
# team listed first will always be the winning team
preprocessed['Pred'] = 1

In [26]:
# rename columns
preprocessed.columns = ['season', 'team_a', 'team_b', 
                        'seed_a', 'seed_b', 
                        'reg_season_wins_a', 'reg_season_wins_b',
                        'reg_season_losses_a', 'reg_season_losses_b',
                        'mean_win_diff_a', 'mean_win_diff_b',
                        'mean_lose_diff_a', 'mean_lose_diff_b',
                        'Pred']

## Prepare for Modeling

In [27]:
import numpy as np
from sklearn.impute import SimpleImputer

In [28]:
# check seed columns for weird values
preprocessed['seed_a'].value_counts()

01    1559
02    1082
03     853
04     703
05     508
06     426
07     388
11     296
08     281
09     249
10     237
12     184
13      81
16      72
14      37
15      25
Name: seed_a, dtype: int64

In [29]:
# convert seed columns to numeric
preprocessed['seed_a'] = preprocessed['seed_a'].astype('int')
preprocessed['seed_b'] = preprocessed['seed_b'].astype('int')

In [30]:
# check column types
preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6981 entries, 0 to 6980
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   season               6981 non-null   int64  
 1   team_a               6981 non-null   int64  
 2   team_b               6981 non-null   int64  
 3   seed_a               6981 non-null   int64  
 4   seed_b               6981 non-null   int64  
 5   reg_season_wins_a    6981 non-null   int64  
 6   reg_season_wins_b    6981 non-null   int64  
 7   reg_season_losses_a  6843 non-null   float64
 8   reg_season_losses_b  6962 non-null   float64
 9   mean_win_diff_a      6981 non-null   float64
 10  mean_win_diff_b      6981 non-null   float64
 11  mean_lose_diff_a     6843 non-null   float64
 12  mean_lose_diff_b     6962 non-null   float64
 13  Pred                 6981 non-null   int64  
dtypes: float64(6), int64(8)
memory usage: 818.1 KB


In [31]:
# for now fill missing values with the column mean
preprocessed['reg_season_losses_a'].fillna(value=preprocessed['reg_season_losses_a'].mean(), inplace=True)
preprocessed['reg_season_losses_b'].fillna(value=preprocessed['reg_season_losses_b'].mean(), inplace=True)
preprocessed['mean_lose_diff_a'].fillna(value=preprocessed['mean_lose_diff_a'].mean(), inplace=True)
preprocessed['mean_lose_diff_b'].fillna(value=preprocessed['mean_lose_diff_b'].mean(), inplace=True)

Our dataset shows significant class imbalance, so we will sample half of the dataframe and present losers first for that half.

In [32]:
# sample datasets to handle class imbalance
df_pre_a = preprocessed.sample(frac=0.5)
df_pre_b = preprocessed.drop(df_pre_a.index)

In [33]:
# relabel one of the dataframes
df_pre_b.columns = ['season', 'team_b', 'team_a',
                    'seed_b', 'seed_a',
                    'reg_season_wins_b', 'reg_season_wins_a',
                    'reg_season_losses_b', 'reg_season_losses_a',
                    'mean_win_diff_b', 'mean_win_diff_a',
                    'mean_lose_diff_b','mean_lose_diff_a',
                    'Pred']

In [34]:
# losers will now be presented first
df_pre_b['Pred'] = 0 

In [35]:
# concat the two back together
df_pre = pd.concat([df_pre_a, df_pre_b], axis=0).reset_index(drop=True)

## Modeling

#### Training and Testing Sets

In [36]:
SEED = 2323

In [37]:
# before creating training and test sets check class imbalance
df_pre['Pred'].value_counts(normalize=True)

0    0.500072
1    0.499928
Name: Pred, dtype: float64

In [38]:
# save a clean copy for modeling
df_clean = df_pre.copy()

In [39]:
# drop team columns
df_clean.drop(['team_a', 'team_b'], axis=1, inplace=True)

In [40]:
# split into training and test sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# identify variables 
y = df_clean['Pred']
X = df_clean.drop('Pred', axis=1)

# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=SEED)

#### Scale Features

In [41]:
# use standard scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

#### Save Final Processed Datasets

In [42]:
X_train_final = X_train_scaled.copy()
X_test_final = X_test_scaled.copy()
y_train_final = y_train.copy()
y_test_final = y_test.copy()

#### Define Function to Print Results

In [43]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_curve, auc
from sklearn import tree

In [44]:
def print_model_scores(X_train, X_test, y_train, y_test, model, model_name):
    """
    Function to return accuracy, recall, precision, f1, roc_auc, and neg_log_loss from given 
    X_train, X_test, y_train, y_test, model, and string for the model name
    """
    
    # create predictions using our model
    y_train_preds = model.predict(X_train)
    y_test_preds = model.predict(X_test)
    
    # accuracy
    train_acc = accuracy_score(y_train, y_train_preds)
    test_acc = accuracy_score(y_test, y_test_preds)
    
    # precision
    train_prec = precision_score(y_train, y_train_preds)
    test_prec = precision_score(y_test, y_test_preds)
    
    # recall 
    train_recall = recall_score(y_train, y_train_preds)
    test_recall = recall_score(y_test, y_test_preds)
    
    # f1 score
    train_f1 = f1_score(y_train, y_train_preds)
    test_f1 = f1_score(y_test, y_test_preds)
    
    # AUC
    train_fpr, train_tpr, train_thresh = roc_curve(y_train, y_train_preds)
    train_roc_auc = auc(train_fpr, train_tpr)
    test_fpr, test_tpr, test_thresh = roc_curve(y_test, y_test_preds)
    test_roc_auc = auc(test_fpr, test_tpr)
    
    # print results
    print('Accuracy:')
    print(f'Training Set: {train_acc}')
    print(f'Testing Set: {test_acc}')
    print('---------------------------')
    print('Precision:')
    print(f'Training Set: {train_prec}')
    print(f'Testing Set: {test_prec}')
    print('---------------------------')
    print('Recall:')
    print(f'Training Set: {train_recall}')
    print(f'Testing Set: {test_recall}')
    print('---------------------------')
    print('F1 Score:')
    print(f'Training Set: {train_f1}')
    print(f'Testing Set: {test_f1}')
    print('---------------------------')
    print(f'ROC AUC:')
    print(f'Training Set: {train_roc_auc}')
    print(f'Test Set: {test_roc_auc}')
    
    # store results in dataframes
    test_results = pd.DataFrame([[f'Test-{model_name}', test_acc, test_prec, test_recall, test_f1, test_roc_auc]],
                                columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC'])
    
    train_results = pd.DataFrame([[f'Training-{model_name}', train_acc, train_prec, train_recall, train_f1, train_roc_auc]],
                                columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC'])
    
    # concat results
    results = pd.concat([test_results, train_results], axis=0)
    
    return results

### XGBoost

In [45]:
from xgboost import XGBClassifier

In [46]:
# create baseline classifier
base_xgb = XGBClassifier(random_state=SEED)

# fit to training set
base_xgb.fit(X_train_final, y_train_final)

# print and store results
base_xgb_results = print_model_scores(X_train_final, X_test_final,
                                      y_train_final, y_test_final,
                                      base_xgb, 'base_xgb')

Accuracy:
Training Set: 0.9902578796561604
Testing Set: 0.7903780068728522
---------------------------
Precision:
Training Set: 0.9889943074003795
Testing Set: 0.7767857142857143
---------------------------
Recall:
Training Set: 0.9916286149162862
Testing Set: 0.8074245939675174
---------------------------
F1 Score:
Training Set: 0.9903097092912788
Testing Set: 0.7918088737201365
---------------------------
ROC AUC:
Training Set: 0.9902523588582198
Test Set: 0.7905901250380575


Without any tuning our model is achieving ~80% accuracy. Given a our class split, a simple model that guessed one class would do substantially worse, with accuracy falling close to 50%. 

#### Tune Hyperparams

In [47]:
# set up gridsearch
grid_search_params = {
    'max_depth': [10, 11, 12],
    'eta': [0.1, 0.05],
    'gamma': [5, 10]
}

# instantiate xgboost
xgb_clf = XGBClassifier()
xgb_grid = GridSearchCV(estimator=xgb_clf,
                        param_grid=grid_search_params,
                        cv=3,
                        scoring='accuracy',
                        return_train_score=True,
                        verbose=1)

# fit to training data
xgb_grid.fit(X_train_final, y_train_final)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             param_grid={'eta': [0.1, 0.05], 'gamma': [5, 10],
                         'max_depth': [10, 11, 12]},
             return_train_score=True, scoring='accuracy', verbose=1)

In [48]:
# print grid search results 
mean_train_score = np.mean(xgb_grid.cv_results_['mean_train_score'])
mean_test_score = np.mean(xgb_grid.cv_results_['mean_test_score'])
print(f'Grid Search Train Accuracy: {mean_train_score}')
print(f'Grid Search Test Accuracy: {mean_test_score}')

Grid Search Train Accuracy: 0.8332378223495702
Grid Search Test Accuracy: 0.7491244826488379


In [49]:
# train model with the best params
xgb_tuned = xgb_grid.best_estimator_
xgb_tuned.fit(X_train_final, y_train_final)
xgb_tuned_results = print_model_scores(X_train_final, X_test_final,
                                       y_train_final, y_test_final,
                                       xgb_tuned, 'xgb_tuned')

Accuracy:
Training Set: 0.8893982808022922
Testing Set: 0.7743413516609393
---------------------------
Precision:
Training Set: 0.8908813429988554
Testing Set: 0.7594235033259423
---------------------------
Recall:
Training Set: 0.8885083713850838
Testing Set: 0.7946635730858469
---------------------------
F1 Score:
Training Set: 0.8896932749095064
Testing Set: 0.7766439909297052
---------------------------
ROC AUC:
Training Set: 0.8894018650174365
Test Set: 0.7745942299818375


Seeing an improvement to overfitting -- we will use this model to create our submissions.

### Ensemble 

In [60]:
xgb_a = XGBClassifier(max_depth=10, gamma=5, eta=0.05, random_state=SEED)
xgb_b = XGBClassifier(max_depth=12, gamma=5, eta=0.05, random_state=SEED)
xgb_c = XGBClassifier(max_depth=8, gamma=5, eta=0.05, random_state=SEED)

xgb_a.fit(X_train_final, y_train_final)
xgb_b.fit(X_train_final, y_train_final)
xgb_c.fit(X_train_final, y_train_final)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.05, eval_metric=None,
              feature_types=None, gamma=5, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, ...)

In [61]:
preds = xgb_tuned.predict_proba(X_test_final)
preds_a = xgb_a.predict_proba(X_test_final)
preds_b = xgb_b.predict_proba(X_test_final)
preds_c = xgb_c.predict_proba(X_test_final)

In [62]:
# ensemble preds
ensemble_preds = np.mean(np.array([preds, preds_a, preds_b, preds_c]), axis=0)
ensemble_preds

array([[0.04320887, 0.95679116],
       [0.7127065 , 0.2872935 ],
       [0.03037937, 0.9696206 ],
       ...,
       [0.4147647 , 0.5852353 ],
       [0.0272468 , 0.97275317],
       [0.15455864, 0.84544134]], dtype=float32)

## Submission

In [63]:
X_train_final.head()

,season,seed_a,seed_b,reg_season_wins_a,reg_season_wins_b,reg_season_losses_a,reg_season_losses_b,mean_win_diff_a,mean_win_diff_b,mean_lose_diff_a,mean_lose_diff_b
0,0.716444,0.506259,-1.220720,-0.369328,2.654827,0.538312,-0.231077,-0.361530,3.815155,0.454249,0.148107
1,-1.169164,-1.236478,-1.220720,1.420648,0.641752,-1.065600,-1.354258,1.636407,-0.023564,1.139414,1.850120
2,1.160116,1.377628,-0.571612,0.397805,0.138483,-0.103253,0.542098,0.914801,0.266922,-1.005900,-0.859567
3,-1.058246,0.506259,2.024821,-1.647882,-0.868054,1.179877,-0.722139,-0.374452,0.364702,-1.940529,-0.956342
4,0.050935,0.506259,0.293866,0.142094,-0.868054,0.217530,1.174216,-0.775715,-0.141055,0.377672,0.389704


In [64]:
# for each ID, create a prediction
first_submission = samp_submission.copy()
first_submission.drop('Pred', axis=1, inplace=True)
first_submission.head()

,ID
0,2023_1101_1102
1,2023_1101_1103
2,2023_1101_1104
3,2023_1101_1105
4,2023_1101_1106


In [65]:
X_train_final.head()

,season,seed_a,seed_b,reg_season_wins_a,reg_season_wins_b,reg_season_losses_a,reg_season_losses_b,mean_win_diff_a,mean_win_diff_b,mean_lose_diff_a,mean_lose_diff_b
0,0.716444,0.506259,-1.220720,-0.369328,2.654827,0.538312,-0.231077,-0.361530,3.815155,0.454249,0.148107
1,-1.169164,-1.236478,-1.220720,1.420648,0.641752,-1.065600,-1.354258,1.636407,-0.023564,1.139414,1.850120
2,1.160116,1.377628,-0.571612,0.397805,0.138483,-0.103253,0.542098,0.914801,0.266922,-1.005900,-0.859567
3,-1.058246,0.506259,2.024821,-1.647882,-0.868054,1.179877,-0.722139,-0.374452,0.364702,-1.940529,-0.956342
4,0.050935,0.506259,0.293866,0.142094,-0.868054,0.217530,1.174216,-0.775715,-0.141055,0.377672,0.389704


In [66]:
# add relevant columns
first_submission['season'] = first_submission['ID'].apply(lambda x: x.split('_')[0]).astype('int64')
first_submission['team_a'] = first_submission['ID'].apply(lambda x: x.split('_')[1]).astype('int64')
first_submission['team_b'] = first_submission['ID'].apply(lambda x: x.split('_')[2]).astype('int64')

In [67]:
# add seed_a information 
seeds['Seed'] = seeds['Seed'].apply(lambda x: re.sub('[^0-9]', '', x)).astype('int64')

In [68]:
# add seed_a to first submission
first_sub = first_submission.merge(seeds, how='left', left_on=['season', 'team_a'], right_on=['Season', 'TeamID'])
first_sub.rename(columns={'Seed':'seed_a'}, inplace=True)
first_sub.drop(['Season', 'TeamID'], axis=1, inplace=True)

In [69]:
# add seed_b to first_submission
first_sub = first_sub.merge(seeds, how='left', left_on=['season', 'team_b'], right_on=['Season', 'TeamID'])
first_sub.rename(columns={'Seed':'seed_b'}, inplace=True)
first_sub.drop(['Season', 'TeamID'], axis=1, inplace=True)

In [70]:
# add reg season wins
wins_group = season_results.groupby(['Season', 'WTeamID']).count().reset_index()
lose_group = season_results.groupby(['Season', 'LTeamID']).count().reset_index()

In [71]:
wins_group = wins_group[['Season', 'WTeamID', 'WScore']]
lose_group = lose_group[['Season', 'LTeamID', 'LScore']]

In [72]:
first_sub = first_sub.merge(wins_group, how='left', left_on=['season', 'team_a'], right_on=['Season', 'WTeamID'])
first_sub.rename(columns={'WScore':'reg_season_wins_a'}, inplace=True)
first_sub.drop(['Season', 'WTeamID'], axis=1, inplace=True)

first_sub = first_sub.merge(wins_group, how='left', left_on=['season', 'team_b'], right_on=['Season', 'WTeamID'])
first_sub.rename(columns={'WScore':'reg_season_wins_b'}, inplace=True)
first_sub.drop(['Season', 'WTeamID'], axis=1, inplace=True)

In [73]:
first_sub = first_sub.merge(lose_group, how='left', left_on=['season', 'team_a'], right_on=['Season', 'LTeamID'])
first_sub.rename(columns={'LScore':'reg_season_losses_a'}, inplace=True)
first_sub.drop(['Season', 'LTeamID'], axis=1, inplace=True)

first_sub = first_sub.merge(lose_group, how='left', left_on=['season', 'team_b'], right_on=['Season', 'LTeamID'])
first_sub.rename(columns={'LScore':'reg_season_losses_b'}, inplace=True)
first_sub.drop(['Season', 'LTeamID'], axis=1, inplace=True)

In [74]:
# add win and lose diffs 
wins_group = season_results.groupby(['Season', 'WTeamID']).mean(numeric_only=True).reset_index()
lose_group = season_results.groupby(['Season', 'LTeamID']).mean(numeric_only=True).reset_index()

In [75]:
wins_group = wins_group[['Season', 'WTeamID', 'WTeamScoreDiff']]
lose_group = lose_group[['Season', 'LTeamID', 'LTeamScoreDiff']]

In [76]:
first_sub = first_sub.merge(wins_group, how='left', left_on=['season', 'team_a'], right_on=['Season', 'WTeamID'])
first_sub.rename(columns={'WTeamScoreDiff':'mean_win_diff_a'}, inplace=True)
first_sub.drop(['Season', 'WTeamID'], axis=1, inplace=True)

first_sub = first_sub.merge(wins_group, how='left', left_on=['season', 'team_b'], right_on=['Season', 'WTeamID'])
first_sub.rename(columns={'WTeamScoreDiff':'mean_win_diff_b'}, inplace=True)
first_sub.drop(['Season', 'WTeamID'], axis=1, inplace=True)

In [77]:
first_sub = first_sub.merge(lose_group, how='left', left_on=['season', 'team_a'], right_on=['Season', 'LTeamID'])
first_sub.rename(columns={'LTeamScoreDiff':'mean_lose_diff_a'}, inplace=True)
first_sub.drop(['Season', 'LTeamID'], axis=1, inplace=True)

first_sub = first_sub.merge(lose_group, how='left', left_on=['season', 'team_b'], right_on=['Season', 'LTeamID'])
first_sub.rename(columns={'LTeamScoreDiff':'mean_lose_diff_b'}, inplace=True)
first_sub.drop(['Season', 'LTeamID'], axis=1, inplace=True)

In [78]:
first_sub.head()

,ID,season,team_a,team_b,seed_a,seed_b,reg_season_wins_a,reg_season_wins_b,reg_season_losses_a,reg_season_losses_b,mean_win_diff_a,mean_win_diff_b,mean_lose_diff_a,mean_lose_diff_b
0,2023_1101_1102,2023,1101,1102,NaN,NaN,9.0,14.0,17.0,18.0,11.333333,12.642857,-11.647059,-10.055556
1,2023_1101_1103,2023,1101,1103,NaN,NaN,9.0,20.0,17.0,11.0,11.333333,15.350000,-11.647059,-11.454545
2,2023_1101_1104,2023,1101,1104,NaN,1.0,9.0,29.0,17.0,5.0,11.333333,18.241379,-11.647059,-12.800000
3,2023_1101_1105,2023,1101,1105,NaN,NaN,9.0,12.0,17.0,18.0,11.333333,9.416667,-11.647059,-11.388889
4,2023_1101_1106,2023,1101,1106,NaN,NaN,9.0,7.0,17.0,23.0,11.333333,7.285714,-11.647059,-15.304348


In [79]:
# fill missing seed values with 1000
first_sub['seed_a'].fillna(value=1000, inplace=True)
first_sub['seed_b'].fillna(value=1000, inplace=True)

In [80]:
# fill everything else with 0s
first_sub.fillna(value=0, inplace=True)
first_sub.isna().sum()

ID                     0
season                 0
team_a                 0
team_b                 0
seed_a                 0
seed_b                 0
reg_season_wins_a      0
reg_season_wins_b      0
reg_season_losses_a    0
reg_season_losses_b    0
mean_win_diff_a        0
mean_win_diff_b        0
mean_lose_diff_a       0
mean_lose_diff_b       0
dtype: int64

In [81]:
first_sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130683 entries, 0 to 130682
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   130683 non-null  object 
 1   season               130683 non-null  int64  
 2   team_a               130683 non-null  int64  
 3   team_b               130683 non-null  int64  
 4   seed_a               130683 non-null  float64
 5   seed_b               130683 non-null  float64
 6   reg_season_wins_a    130683 non-null  float64
 7   reg_season_wins_b    130683 non-null  float64
 8   reg_season_losses_a  130683 non-null  float64
 9   reg_season_losses_b  130683 non-null  float64
 10  mean_win_diff_a      130683 non-null  float64
 11  mean_win_diff_b      130683 non-null  float64
 12  mean_lose_diff_a     130683 non-null  float64
 13  mean_lose_diff_b     130683 non-null  float64
dtypes: float64(10), int64(3), object(1)
memory usage: 15.0+ MB


In [82]:
# pull out relevant columns and scale
X_submission = first_sub.drop(['ID', 'team_a', 'team_b'], axis=1)
scaler = StandardScaler()

X_sub_scaled = scaler.fit_transform(X_submission)
X_sub_scaled = pd.DataFrame(X_sub_scaled, columns=X_submission.columns)

In [83]:
# create dataset to create preds
X_sub_final = X_sub_scaled.copy()
X_sub_final.head()

,season,seed_a,seed_b,reg_season_wins_a,reg_season_wins_b,reg_season_losses_a,reg_season_losses_b,mean_win_diff_a,mean_win_diff_b,mean_lose_diff_a,mean_lose_diff_b
0,0.0,0.482644,0.479194,-0.943261,-0.206847,0.313101,0.540784,-0.211913,0.143378,0.219635,0.612320
1,0.0,0.482644,0.479194,-0.943261,0.756535,0.313101,-0.741464,-0.211913,0.797918,0.219635,0.251174
2,0.0,0.482644,-2.107070,-0.943261,2.201607,0.313101,-1.840534,-0.211913,1.497003,0.219635,-0.096151
3,0.0,0.482644,0.479194,-0.943261,-0.527974,0.313101,0.540784,-0.211913,-0.636659,0.219635,0.268123
4,0.0,0.482644,0.479194,-0.943261,-1.330792,0.313101,1.456676,-0.211913,-1.151886,0.219635,-0.742642


In [84]:
print(len(X_sub_final.columns))
print(len(X_train_final.columns))

11
11


In [85]:
# generate predictions
submission_preds = xgb_tuned.predict_proba(X_sub_final)

In [87]:
xgb_tuned.classes_

array([0, 1])

In [88]:
# convert to a dataframe
submission = pd.DataFrame(submission_preds, columns=['loss_prob', 'win_prob'])
submission.head()

,loss_prob,win_prob
0,0.561315,0.438685
1,0.455919,0.544081
2,0.942501,0.057499
3,0.377383,0.622617
4,0.495534,0.504466


In [89]:
# drop loss_prob
submission = submission.drop('loss_prob', axis=1)
submission.columns = ['Pred']
submission.head()

,Pred
0,0.438685
1,0.544081
2,0.057499
3,0.622617
4,0.504466


In [90]:
final_submission = first_sub[['ID']]
final_submission = pd.concat([final_submission, submission], axis=1)
final_submission.head()

,ID,Pred
0,2023_1101_1102,0.438685
1,2023_1101_1103,0.544081
2,2023_1101_1104,0.057499
3,2023_1101_1105,0.622617
4,2023_1101_1106,0.504466


In [91]:
submit_path = Path('../submissions/')
final_submission.to_csv(submit_path/'akg_submission_1.csv', index=False)

## Ensemble Predictions

In [92]:
# generate predictions from submission file
sub_preds_1 = xgb_tuned.predict_proba(X_sub_final)
sub_preds_2 = xgb_a.predict_proba(X_sub_final)
sub_preds_3 = xgb_b.predict_proba(X_sub_final)
sub_preds_4 = xgb_c.predict_proba(X_sub_final)

# ensemble together using bagging
ensemble_preds = np.mean(np.array([sub_preds_1, sub_preds_2, sub_preds_3, sub_preds_4]), axis=0)

In [99]:
ens_submission = pd.DataFrame(ensemble_preds, columns=['loss_prob', 'win_prob'])

# drop loss_prob
ens_submission = ens_submission.drop('loss_prob', axis=1)
ens_submission.columns = ['Pred']
ens_submission.head()

,Pred
0,0.458548
1,0.538600
2,0.059626
3,0.594755
4,0.485791


In [100]:
ensemble_submission = first_sub[['ID']]
ensemble_submission = pd.concat([ensemble_submission, ens_submission], axis=1)
ensemble_submission.head()

,ID,Pred
0,2023_1101_1102,0.458548
1,2023_1101_1103,0.538600
2,2023_1101_1104,0.059626
3,2023_1101_1105,0.594755
4,2023_1101_1106,0.485791


In [101]:
# save to CSV
final_submission.to_csv(submit_path/'akg_ensemble_submission.csv', index=False)

## Information for Bracket

In [102]:
df = ensemble_submission.copy()
df['Season'] = df['ID'].apply(lambda x: x.split('_')[0]).astype('int64')
df['team_a'] = df['ID'].apply(lambda x: x.split('_')[1]).astype('int64')
df['team_b'] = df['ID'].apply(lambda x: x.split('_')[2]).astype('int64')

In [103]:
team_names = teams.drop(['FirstD1Season', 'LastD1Season'], axis=1)

# merge into df
df = df.merge(team_names, how='left', left_on='team_a', right_on='TeamID')
df.rename(columns={'TeamName':'team_a_name'}, inplace=True)
df.drop('TeamID', axis=1, inplace=True)

df = df.merge(team_names, how='left', left_on='team_b', right_on='TeamID')
df.rename(columns={'TeamName':'team_b_name'}, inplace=True)
df.drop('TeamID', axis=1, inplace=True)

In [104]:
df.head()

,ID,Pred,Season,team_a,team_b,team_a_name,team_b_name
0,2023_1101_1102,0.458548,2023,1101,1102,Abilene Chr,Air Force
1,2023_1101_1103,0.538600,2023,1101,1103,Abilene Chr,Akron
2,2023_1101_1104,0.059626,2023,1101,1104,Abilene Chr,Alabama
3,2023_1101_1105,0.594755,2023,1101,1105,Abilene Chr,Alabama A&M
4,2023_1101_1106,0.485791,2023,1101,1106,Abilene Chr,Alabama St


In [105]:
bracket_df = df[['team_a', 'team_a_name', 'Pred', 'team_b', 'team_b_name']]
bracket_df.columns = ['team_a_ID', 'team_a_name', 'a_win_prob', 'team_b_ID', 'team_b_name']

In [106]:
bracket_df.head()

,team_a_ID,team_a_name,a_win_prob,team_b_ID,team_b_name
0,1101,Abilene Chr,0.458548,1102,Air Force
1,1101,Abilene Chr,0.538600,1103,Akron
2,1101,Abilene Chr,0.059626,1104,Alabama
3,1101,Abilene Chr,0.594755,1105,Alabama A&M
4,1101,Abilene Chr,0.485791,1106,Alabama St


In [107]:
# save bracket_df as csv
bracket_df.to_csv(data_path/'bracket_probs.csv', index=False)

In [108]:
# create a function to look up a matchup
def lookup_matchup(team_a, team_b, data=bracket_df):
    return data.loc[(data['team_a_name'] == team_a) 
                    & (data['team_b_name'] == team_b)]

In [188]:
lookup_matchup("Creighton", "Houston")

,team_a_ID,team_a_name,a_win_prob,team_b_ID,team_b_name
19701,1166,Creighton,0.337236,1222,Houston
84984,3166,Creighton,0.863959,3222,Houston
